In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
# Read in data
# products = pd.read_csv("../Resources/products.csv")
# accounts = pd.read_csv("../Resources/accounts.csv")
ast = pd.read_csv("../Resources/account_state_transitions.csv")
# gpa = pd.read_csv("../Resources/group_product_associations.csv")
# groups = pd.read_csv("../Resources/groups.csv")
# payments = pd.read_csv("../Resources/payments.csv")

In [3]:
ast.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

In [ ]:
# State as of: date (Using 'started_when' for now)

# SELECT 'to_state' WHERE 'started_when' == timestamp

In [45]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-05


In [46]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

In [47]:
# Change data type from string to datetime object
ast['started_when'] = pd.to_datetime(ast['started_when'])

In [48]:
# Create limited dataframe based on indicated timestamp 
df = ast.loc[(ast['started_when'] >= timestamp_dt) & (ast['started_when'] < timestamp_dt + timedelta(days=1))]
df.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
86,443,2020-03-12 20:03:41.980224,2020-03-05 01:00:17.571392,5,DISABLED,ENABLED,442.0
474,3049,2020-03-12 20:04:21.579794,2020-03-05 15:06:23.571392,20,DISABLED,ENABLED,3048.0
651,3822,2020-03-12 20:04:40.010129,2020-03-05 08:38:12.571392,25,ENABLED,DISABLED,3821.0
938,5417,2020-03-12 20:05:20.780819,2020-03-05 10:22:37.879084,35,ENABLED,DISABLED,5416.0
1666,10661,2020-03-12 20:07:26.635161,2020-03-05 07:08:35.571392,72,ENABLED,DISABLED,10660.0


Try timestamp input with time HH:MM:SS

Check timedelta(days=1) -> 24hrs? or before midnight (11:59)

Need to select "to_state" based on timestamp, then calculate timedelta based on Peter's answer